In [1]:
from datasets import load_dataset, Features, ClassLabel, Value, Sequence
from transformers import AutoTokenizer
import os
from pathlib import Path

## Tokenizer

In [2]:
# Make sure the tokenizer match the model that will be fine-tuned.
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Load the datasets
This cell loads the generated `train.jsonl`, `validation.jsonl`, and `test.jsonl` files.

In [3]:
project_root = str(Path().cwd().resolve().parent)
dataset_dir = os.path.join(project_root, "data", "raw")

data_files = {
    "train": os.path.join(dataset_dir, "train.jsonl"),
    "validation": os.path.join(dataset_dir, "validation.jsonl"),
    "test": os.path.join(dataset_dir, "test.jsonl")
}
raw_datasets = load_dataset("json", data_files=data_files)

print(raw_datasets)
print("\nExample from training set:")
print(raw_datasets['train'][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'entities'],
        num_rows: 1864
    })
    validation: Dataset({
        features: ['text', 'intent', 'entities'],
        num_rows: 400
    })
    test: Dataset({
        features: ['text', 'intent', 'entities'],
        num_rows: 400
    })
})

Example from training set:
{'text': 'I must cancel my sabbatical leave meeting with Wright.', 'intent': 'cancel', 'entities': [{'start': 17, 'end': 41, 'label': 'appointment_type'}, {'start': 47, 'end': 53, 'label': 'practitioner_name'}]}


## Create Label Mappings
Now, we create the mappings from string labels (e.g., "schedule", "practitioner_name") to integer IDs. This is essential for training. We also need to create tags for the BIO (Beginning, Inside, Outside) entity scheme.

In [4]:
# --- Create Intent Label Mappings ---
# Get all unique intent labels from the training data
intent_labels = raw_datasets['train'].unique('intent')
intent_labels.sort() # Sort for consistency
id2intent = {i: label for i, label in enumerate(intent_labels)}
intent2id = {label: i for i, label in enumerate(intent_labels)}
print(f"Intent mapping (intent2id): {intent2id}\n")


# --- Create Entity (NER) Label Mappings in BIO format ---
# Get all unique entity labels
entity_labels = ["appointment_id", "appointment_type", "practitioner_name"]
# Create the full list of BIO tags
ner_tags = ["O"] # 'O' for tokens outside any entity
for label in entity_labels:
    ner_tags.append(f"B-{label}") # 'B' for Beginning of an entity
    ner_tags.append(f"I-{label}") # 'I' for Inside of an entity

id2ner = {i: label for i, label in enumerate(ner_tags)}
ner2id = {label: i for i, label in enumerate(ner_tags)}
print(f"NER mapping (ner2id): {ner2id}")

Intent mapping (intent2id): {'bye': 0, 'cancel': 1, 'greeting': 2, 'negative_reply': 3, 'oos': 4, 'positive_reply': 5, 'query_avail': 6, 'reschedule': 7, 'schedule': 8}

NER mapping (ner2id): {'O': 0, 'B-appointment_id': 1, 'I-appointment_id': 2, 'B-appointment_type': 3, 'I-appointment_type': 4, 'B-practitioner_name': 5, 'I-practitioner_name': 6}


## Preprocessing function
This is the core function. It takes a single data example and does two things:
1. Tokenizes the text.
2. Aligns character-based entity spans (`start`, `end`) with the new wordpiece tokens, assigning the correct BIO tag ID to each token.

In [5]:
def preprocess_function(examples):
    # --- Intent Processing ---
    # Convert intent strings to integer IDs
    intent_ids = [intent2id[intent] for intent in examples['intent']]

    # --- Tokenization ---
    # Tokenize the text. `truncation=True` and `padding` are handled by the Trainer later.
    tokenized_inputs = tokenizer(examples['text'], truncation=True, is_split_into_words=False, return_offsets_mapping=True)

    # --- Entity (NER) Label Alignment ---
    ner_labels = []
    for i, entities in enumerate(examples['entities']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        # Start with all tokens labeled as 'O' (Outside)
        label_ids = [ner2id["O"]] * len(word_ids)
        
        # For each entity, find the corresponding tokens and assign B- and I- tags
        for entity in entities:
            start_char, end_char, label = entity['start'], entity['end'], entity['label']
            
            for j, word_id in enumerate(word_ids):
                if word_id is None:
                    continue
                
                # Get the character span for the current token
                token_char_span = tokenized_inputs['offset_mapping'][i][j]
                if token_char_span is None:
                    continue

                token_start, token_end = token_char_span
                
                # Check if the token is part of the entity
                if start_char < token_end and end_char > token_start:
                    if label_ids[j] == ner2id["O"]:
                        # Assign the 'B-' tag to the first token
                        label_ids[j] = ner2id[f"B-{label}"]
                    else:
                        # Assign the 'I-' tag to subsequent tokens within the same entity
                        label_ids[j] = ner2id[f"I-{label}"]

        ner_labels.append(label_ids)

    # Add the final processed labels to our tokenized inputs
    tokenized_inputs["intent_label"] = intent_ids
    tokenized_inputs["ner_labels"] = ner_labels
    
	# Remove offset_mapping
    tokenized_inputs.pop("offset_mapping", None)
    
    return tokenized_inputs

## Apply Preprocessing and Save
Now we apply this function to our entire dataset and save the final, processed version. This is what we will load directly in the fine-tuning script.

In [6]:
# Apply the function to all splits of the dataset
processed_datasets = raw_datasets.map(preprocess_function, batched=True, remove_columns=raw_datasets['train'].column_names)

# Define the features for our processed dataset, including the new ClassLabels
features = Features({
    'input_ids': Sequence(Value('int64')),
    'attention_mask': Sequence(Value('int8')),
    'intent_label': ClassLabel(names=list(intent2id.keys())),
    'ner_labels': Sequence(ClassLabel(names=list(ner2id.keys())))
})

# Cast the processed datasets to the defined features to include the label names
processed_datasets = processed_datasets.cast(features)

# Save the processed dataset locally
output_dir = os.path.join(project_root, "data", "processed", "hasd_processed")
processed_datasets.save_to_disk(output_dir)

print("\nProcessed dataset saved successfully!")
print(processed_datasets)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1864 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/400 [00:00<?, ? examples/s]


Processed dataset saved successfully!
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'intent_label', 'ner_labels'],
        num_rows: 1864
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'intent_label', 'ner_labels'],
        num_rows: 400
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'intent_label', 'ner_labels'],
        num_rows: 400
    })
})
